Import libraries

In [1]:
#data loading and manipulation
import pandas as pd
import numpy as np

#for working with file paths
from os import path

#regular expressions
import re

#punctuation
import string

#spelling
#import enchant

In [2]:
pd.set_option('display.max_rows', 1000)

Load exempt data
* AQ06_where_data.csv is AQ06s submitted on or after 08/01/2016 and before 09/01/2016
* ANF_where_data.csv is ANFs submitted on or after 08/01/2016 and before 09/01/2016

In [3]:
aq06_path = path.join('data', 'AQ06_where_data.csv')
anf_path = path.join('data', 'ANF_where_data.csv')

aq06 = pd.read_csv(aq06_path)
anf = pd.read_csv(anf_path)

In [4]:
print aq06.shape
aq06.head()

(158, 7)


,ADDRESS,FORMTYPE,STICKER.,CONTRACTOR,STARTDATE,ENDDATE,EXPORTDATE
0,"154-160 GROVE STREET, CHICOPEE, MA",AQ-06,100211920R6,"COSTELLO DISMANTLING COMPANY, INC.",12/29/2014 0:00:00,10/30/2016 0:00:00,12/13/2016 11:30:59
1,451 BOSTON AVE (MEDFORD CONFIRMING THIS ADRESS...,AQ-06,100214663R1,"O'CONNOR CONSTRUCTORS, INC.",2/12/2015 0:00:00,5/31/2017 0:00:00,12/13/2016 11:30:59
2,"191 PARK STREET, NORTH READING, MA",AQ-06,100223260R2,"NASDI, LLC",7/6/2015 0:00:00,8/18/2016 0:00:00,12/13/2016 11:30:59
3,"103 MARBLE ST, SPRINGFIELD, MA",AQ-06,100228296,"FONTAINE BROS., INC",9/1/2016 0:00:00,5/1/2018 0:00:00,12/13/2016 11:30:59
4,"60 HODGES AVENUE, TAUNTON, MA",AQ-06,100233015R1,BOND BROTHERS,12/1/2015 0:00:00,12/31/2016 0:00:00,12/13/2016 11:30:59


In [5]:
print anf.shape
anf.head()

(2262, 7)


,ADDRESS,FORMTYPE,STICKER.,CONTRACTOR,STARTDATE,ENDDATE,EXPORTDATE
0,"70 CIRCUIT ROAD, BROOKLINE, MA",ANF-001,100249232,ENVIROGREEN LLC,8/30/2016 0:00:00,8/31/2016 0:00:00,12/13/2016 11:30:26
1,"24 BEACON STREET, BOSTON, MA",ANF-001,100247424R1,ENCORE CONTRACTING SERVICES INC,8/18/2016 0:00:00,8/29/2016 0:00:00,12/13/2016 11:30:26
2,"21 COLLEGE LANE, NORTHAMPTON, MA",ANF-001,100249107,ABIDE INC,8/16/2016 0:00:00,8/16/2016 0:00:00,12/13/2016 11:30:26
3,"140 PINE ST, HYANNIS, MA",ANF-001,100246632R2,AIR SAFE INC,8/22/2016 0:00:00,8/22/2016 0:00:00,12/13/2016 11:30:26
4,"85 PROVIDENCE HIGHWAY, WESTWOOD, MA",ANF-001,100249263,ENVIRONMENTAL RESPONSE SERVICES INC,8/30/2016 0:00:00,9/27/2016 0:00:00,12/13/2016 11:30:26


Check for and reconcile missing data

In [6]:
aq06.isnull().sum()

ADDRESS       0
FORMTYPE      0
STICKER.      0
CONTRACTOR    0
STARTDATE     0
ENDDATE       0
EXPORTDATE    0
dtype: int64

In [7]:
aq06[aq06.FORMTYPE.isnull()]

,ADDRESS,FORMTYPE,STICKER.,CONTRACTOR,STARTDATE,ENDDATE,EXPORTDATE


In [8]:
def replace_null(df,col_name,value):
    df[col_name] = df[col_name].fillna(value)

replace_null(aq06,'FORMTYPE','AQ-06')

In [9]:
anf.isnull().sum()

ADDRESS       0
FORMTYPE      0
STICKER.      0
CONTRACTOR    0
STARTDATE     0
ENDDATE       0
EXPORTDATE    0
dtype: int64

In [10]:
open_today = pd.DataFrame()
open_today = open_today.append(aq06)
open_today = open_today.append(anf)
open_today.shape

(2420, 7)

In [11]:
#check for SAME
print open_today.loc[open_today['CONTRACTOR']=='SAME']

Empty DataFrame
Columns: [ADDRESS, FORMTYPE, STICKER., CONTRACTOR, STARTDATE, ENDDATE, EXPORTDATE]
Index: []


In [12]:
#check FORMTYPE values
print open_today['FORMTYPE'].value_counts()

ANF-001    2262
AQ-06       156
06            2
Name: FORMTYPE, dtype: int64


In [13]:
#clean up FORMTYPE
open_today.loc[open_today['FORMTYPE'].str.contains('06'),'FORMTYPE']='aq06'
open_today.loc[open_today['FORMTYPE'].str.contains('001'),'FORMTYPE']='anf001'

Remove punctuation from all text fields and convert to uppercase

In [14]:
#copy CONTRACTOR to CON to preserve original name; all edtis to be done on CON
for row in open_today:
    open_today['CON'] = open_today['CONTRACTOR']

In [15]:
#count unique CON
print len(open_today['CON'].value_counts())
open_today['CON'].value_counts()

236


ABIDE INC                                         99
FJC ENTERPRISES INC                               99
DEC-TAM CORPORATION                               97
AIR SAFE  INC                                     70
NORTHSTAR CONTRACTING GROUP INC                   63
NON LICENSED REMOVAL                              58
NEW ROADS ENVIRONMENTAL SERVICES LLC              54
BAYSTATE CONTRACTING SERVICES  INC                52
S & S ABATEMENT LLC                               45
GREEN ENVIRONMENTAL                               45
WALSH ENVIRONMENTAL SERVICES                      44
CLEAN AIR ENVIRONMENTAL INC                       43
NATIONAL ABATEMENT INC                            43
ENVI LABOR SERVIVES INC                           42
ASBESTOS FREE  INC                                42
A & E ENVIRONMENTAL INC                           42
BANNER ENVIRONMENTAL SERVICES INC                 42
COMPASS RESTORATION SERVICE SERVICES  LLC         38
ENVIRONMENTAL RESPONSE SERVICES  INC          

In [16]:
def upper_no_punct(df,col_name):
    for field in df[col_name]:
        new_field = field.translate(None, string.punctuation).upper()
        df[col_name] = df[col_name].replace(field,new_field)

upper_no_punct(open_today,'CON')
#370-->341 aq06 only
#502-->463 aq06 + anf

In [17]:
#Replaces 'pattern' in 'string' with 'sub' if 'pattern' starts 'string'
def lreplace(pattern, sub, string):
    return re.sub('^%s' % pattern, sub, string)

#Replaces 'pattern' in 'string' with 'sub' if 'pattern' ends 'string'
def rreplace(pattern, sub, string):
    return re.sub('%s$' % pattern, sub, string)

In [18]:
#standardize abbreviations and misspellings
def remove_abbrev(df, col_name, old_string, new_string):
    for field in df[col_name]:
        new_field = field.replace(old_string,new_string)
        df[col_name] = df[col_name].replace(field,new_field)

#remove abbreviations that appear at the end of the string only (ex: CO at the end, but not CO in CONSTRUCTION)
def remove_ending(df, col_name, ending):
    for field in df.loc[df[col_name].str.endswith(ending),col_name]:
        new_field = rreplace(ending,'',field)
        new_field = new_field.rstrip()
        df[col_name] = df[col_name].replace(field,new_field)

In [19]:
#slower?
abbrevs = {
    'AND':('ADN',),
    'BRO':('BROS',
           'BROTHER',
           'BROTHERS'),
    'BUILDING':('BUIDLING',
                'BUILIDNG'),
    'CAULFIELD':('CAULLFIELD',),
    'CONSTRUCTION':('CONSTRCTION',
                    'CONSTRCUTION',
                    'CONSTRUCION',
                    'CONSTRUCTON',
                    'CONSTRUCTUON',
                    'CONSTRUTION',
                    'CONTRUCTION',),
    'CORP':('CORPORATION',
            'CORPORATIO',
            'CORPOTATION',),
    'DESIGN':('DEISGN',),
    'DEMO':('DEMOLIITION',
            'DEMOLITION',
            'DEMOLTION'),
    'ENVIRONMENTAL':('ENVIRONMENATL',
                    'ENVIRONMETAL'),
    'INC':('INCORPORATED',),
    'JK SCANLAN':('JKS',),
    'SHAWMUT':('SAHWMUT',),
    'STRUCTURETONE':('STRUCTURE TONE',
                    'STURUCTURE TONE',),
    'SON':('SONS',),
    'DISMANTLING':('DISMANTLIING',),
}

spacing = {
    ' ':('  ',),
}

endings = [
    'LLC',
    'COINC',
    'INC',
    'COMPANY',
    'CORP',
    'LLP',
    'DESIGN',
    'CO',
    'CONC',
    'MANAGERS',
    'GROUP',
    'SERVICES',
    'SERVICE',
    'AND',
]

for new_string, old_strings in abbrevs.iteritems():
    for old_string in old_strings: 
        remove_abbrev(open_today, 'CON', old_string, new_string)

for ending in endings:
        remove_ending(open_today, 'CON', ending)
        
for new_string, old_strings in spacing.iteritems():
    for old_string in old_strings: 
        remove_abbrev(open_today, 'CON', old_string, new_string)
#341-->273 aq06 only
#463--382 aq06 + anf

In [20]:
#count unique CON
print len(open_today['CON'].value_counts())
open_today['CON'].value_counts()

213


ABIDE                                       99
FJC ENTERPRISES                             99
DECTAM                                      97
AIR SAFE                                    70
NORTHSTAR CONTRACTING                       63
NON LICENSED REMOVAL                        58
NEW ROADS ENVIRONMENTAL                     54
BAYSTATE CONTRACTING                        52
S S ABATEMENT                               45
GREEN ENVIRONMENTAL                         45
WALSH ENVIRONMENTAL                         44
CLEAN AIR ENVIRONMENTAL                     43
NATIONAL ABATEMENT                          43
ASBESTOS FREE                               42
BANNER ENVIRONMENTAL                        42
A E ENVIRONMENTAL                           42
ENVI LABOR SERVIVES                         42
THE AULSON                                  41
COMPASS RESTORATION                         38
ENVIRONMENTAL RESPONSE                      37
ECONOMIC ENVIRO TECHS                       35
NORTHEAST REM

#faster?
abbrevs = {
    'ADN':'AND',
    'BROS':'BRO',
    'BROTHER':'BRO',
    'BROTHERS':'BRO',
    'BUIDLING':'BUILDING',
    'BUILIDNG':'BUILDING',
    'CAULLFIELD':'CAULFIELD',
    'CONSTRCTION':'CONSTRUCTION',
    'CONSTRCUTION':'CONSTRUCTION',
    'CONSTRUCION':'CONSTRUCTION',
    'CONSTRUCTON':'CONSTRUCTION',
    'CONSTRUCTUON':'CONSTRUCTION',
    'CONSTRUTION':'CONSTRUCTION',
    'CONTRUCTION':'CONSTRUCTION',
    'CORPORATION':'CORP',
    'CORPORATIO':'CORP',
    'CORPOTATION':'CORP',
    'DEISGN':'DESIGN',
    'DEMOLIITION':'DEMO',
    'DEMOLITION':'DEMO',
    'DEMOLTION':'DEMO',
    'DISMANTLIING':'DISMANTLING',
    'ENVIRONMENATL':'ENVIRONMENTAL',
    'ENVIRONMETAL':'ENVIRONMENTAL',
    'INCORPORATED':'INC',
    'JKS':'JK SCANLAN',
    'SAHWMUT':'SHAWMUT',
    'STRUCTURE TONE':'STRUCTURETONE',
    'STURUCTURE TONE':'STRUCTURETONE',
    'SONS':'SON',    
}

spacing = {
    '  ':' ',
}

endings = [
    'LLC',
    'COINC',
    'INC',
    'COMPANY',
    'CORP',
    'LLP',
    'DESIGN',
    'CO',
    'CONC',
    'MANAGERS',
    'GROUP',
    'SERVICES',
    'SERVICE',
    'AND',
]

for old_string, new_string in abbrevs.iteritems():
    remove_abbrev(open_today, 'CON', old_string, new_string)

for ending in endings:
    remove_ending(open_today, 'CON', ending)
        
for old_string, new_strings in spacing.iteritems():
    remove_abbrev(open_today, 'CON', old_string, new_string)
#341-->273 aq06 only
#463--382 aq06 + anf

#standardize abbreviations and misspellings using using re.sub
def re_remove_abbrev(df, col_name, old_string, new_string):
    for field in df[col_name]:
        new_field = re.sub(old_string, new_string, old_string)
        df[col_name] = df[col_name].replace(field,new_field)

#fastest?
abbrevs = {
    'ADN':'AND',
    'BROS':'BRO',
    'BROTHER':'BRO',
    'BROTHERS':'BRO',
    'BUIDLING':'BUILDING',
    'BUILIDNG':'BUILDING',
    'CAULLFIELD':'CAULFIELD',
    'CONSTRCTION':'CONSTRUCTION',
    'CONSTRCUTION':'CONSTRUCTION',
    'CONSTRUCION':'CONSTRUCTION',
    'CONSTRUCTON':'CONSTRUCTION',
    'CONSTRUCTUON':'CONSTRUCTION',
    'CONSTRUTION':'CONSTRUCTION',
    'CONTRUCTION':'CONSTRUCTION',
    'CORPORATION':'CORP',
    'CORPORATIO':'CORP',
    'CORPOTATION':'CORP',
    'DEISGN':'DESIGN',
    'DEMOLIITION':'DEMO',
    'DEMOLITION':'DEMO',
    'DEMOLTION':'DEMO',
    'DISMANTLIING':'DISMANTLING',
    'ENVIRONMENATL':'ENVIRONMENTAL',
    'ENVIRONMETAL':'ENVIRONMENTAL',
    'INCORPORATED':'INC',
    'JKS':'JK SCANLAN',
    'SAHWMUT':'SHAWMUT',
    'STRUCTURE TONE':'STRUCTURETONE',
    'STURUCTURE TONE':'STRUCTURETONE',
    'SONS':'SON',    
}

spacing = {
    '  ':' ',
}

endings = [
    'LLC',
    'COINC',
    'INC',
    'COMPANY',
    'CORP',
    'LLP',
    'DESIGN',
    'CO',
    'CONC',
    'MANAGERS',
    'GROUP',
    'SERVICES',
    'SERVICE',
    'AND',
]

for old_string, new_string in abbrevs.iteritems():
    re_remove_abbrev(open_today, 'CON', old_string, new_string)

#for ending in endings:
#    remove_ending(open_today, 'CON', ending)
        
#for old_string, new_strings in spacing.iteritems():
#    remove_abbrev(open_today, 'CON', old_string, new_string)
#341-->273 aq06 only
#463--382 aq06 + anf

In [21]:
#write data to csv
open_today.to_csv('where.csv', columns=('ADDRESS','FORMTYPE','STICKER.','CON','STARTDATE','ENDDATE','EXPORTDATE'), header=False, index=False)

In [22]:
con = open_today[open_today['CON'].str.contains('NASDI')]
print con['FORMTYPE'].value_counts()

anf001    7
aq06      2
Name: FORMTYPE, dtype: int64
